In [1]:
# Import torch & Check CUDA availability
import torch

print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.current_device())

True
1
0


In [2]:
# Get CUDA device name
print(torch.cuda.device(0))
print(torch.cuda.get_device_name(0))

NVIDIA A30


In [3]:
# Import dataset from saved PyTorch dataset
data = torch.load("/dfs6/pub/seminl1/graph500/graph500_scale24_ef64.pt")

# Store the dataset to GPU
#data = data.pin_memory()
#data = data.to('cuda:0', non_blocking=True)

In [4]:
# Check whether the dataset is stored on the GPU or not
print(f'Graph is stored on the GPU: {data.is_cuda}')

Graph is stored on the GPU: False


In [5]:
# Print first element
print(f'Graph: {data}')

Graph: Data(x=[16777216, 64], edge_index=[2, 1044447191], y=[16777216], train_mask=[16777216])


In [6]:
# Node feature matrix information
print(f'x = {data.x.shape}')
print(data.x)

x = torch.Size([16777216, 64])
tensor([[ 1.7491,  1.0508, -1.1073,  ..., -1.6631,  0.4232, -1.6230],
        [ 1.4225, -1.3486,  0.0630,  ..., -0.3219, -1.0920, -0.3329],
        [ 0.6128, -1.0707, -1.4233,  ...,  0.4200,  0.1730,  1.0194],
        ...,
        [ 0.5497, -0.4471,  1.0664,  ...,  0.7018,  0.1765, -1.6260],
        [ 2.3088,  1.4693, -1.4589,  ..., -2.0359,  2.3048,  1.3808],
        [-0.1230,  1.0694,  2.1998,  ..., -1.5174,  0.5692, -1.5635]])


In [7]:
# Edge index information
print(f'edge_index = {data.edge_index.shape}')
print(data.edge_index)

edge_index = torch.Size([2, 1044447191])
tensor([[       0,        0,        0,  ..., 16777215, 16777215, 16777215],
        [       0,        1,        2,  ..., 16147194, 16285027, 16745422]])


In [8]:
# Train mask
print(f'train_mask = {data.train_mask.shape}')
print(data.train_mask)

train_mask = torch.Size([16777216])
tensor([False,  True, False,  ..., False,  True, False])


In [9]:
# Try to get the number of addition operation
start_point = 0
numAddition = 0
numEdges = 1044447191
for i in range(numEdges-1):
    if data.edge_index[0][i+1] == start_point:
        numAddition += 1
    else:
        start_point = data.edge_index[0][i+1]
        if data.edge_index[0][i] % 100000 == 0:
            print(data.edge_index[0][i])
print(numAddition)

tensor(0)
tensor(200000)
tensor(400000)
tensor(600000)
tensor(700000)
tensor(800000)
tensor(900000)
tensor(1000000)
tensor(1100000)
tensor(1200000)
tensor(1300000)
tensor(1400000)
tensor(1500000)
tensor(1600000)
tensor(1700000)
tensor(1800000)
tensor(1900000)
tensor(2000000)
tensor(2200000)
tensor(2300000)
tensor(2400000)
tensor(2500000)
tensor(2600000)
tensor(2700000)
tensor(2800000)
tensor(3000000)
tensor(3100000)
tensor(3400000)
tensor(3500000)
tensor(3600000)
tensor(3700000)
tensor(3800000)
tensor(3900000)
tensor(4000000)
tensor(4100000)
tensor(4200000)
tensor(4300000)
tensor(4400000)
tensor(4500000)
tensor(4600000)
tensor(4800000)
tensor(5000000)
tensor(5100000)
tensor(5200000)
tensor(5400000)
tensor(5500000)
tensor(5600000)
tensor(5700000)
tensor(5800000)
tensor(5900000)
tensor(6200000)
tensor(6300000)
tensor(6400000)
tensor(6800000)
tensor(6900000)
tensor(7000000)
tensor(7100000)
tensor(7200000)
tensor(7300000)
tensor(7500000)
tensor(7800000)
tensor(8000000)
tensor(8100000)
tens

In [ ]:
# Ground-truth labels
print(f'y = {data.y.shape}')
print(data.y)

In [ ]:
from torch_geometric.loader import NeighborLoader

# Create batches with neighbor sampling
train_loader = NeighborLoader(
    data,
    num_neighbors=[30],
    batch_size=2048,
    input_nodes=data.train_mask,
)

In [ ]:
# Create a simple GCN with only one GCN layer
import torch.nn.functional as F

from torch.nn import Linear
from torch_geometric.nn import GCNConv

class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.gcn1 = GCNConv(64, 64)
        self.optimizer = torch.optim.Adam(self.parameters(),
                                          lr=0.01,
                                          weight_decay=5e-4)

    def forward(self, x, edge_index):
        x = self.gcn1(x, edge_index)
        z = F.log_softmax(x, dim=1)
        return x, z

In [ ]:
def accuracy(pred_y, y):
    """Calculate accuracy."""
    return ((pred_y == y).sum() / len(y)).item()

def train(model, data, train_loader, device):
    """Train a GNN model and return the trained model."""
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = model.optimizer
    epochs = 200
    total_elapsed_time = 0
    
    model.train()
    for epoch in range(epochs+1):
        # Training
        total_loss = 0
        acc = 0
        val_loss = 0
        val_acc = 0
        
        # Train on batches
        for batch in train_loader:
            batch = batch.to(device)
            optimizer.zero_grad()
            h, out = model(batch.x, batch.edge_index)
            loss = criterion(out[batch.train_mask], batch.y[batch.train_mask])
            total_loss += loss
            acc += accuracy(out[batch.train_mask].argmax(dim=1), batch.y[batch.train_mask])
            loss.backward()
            optimizer.step()
                
        # Print metrics every 10 epochs
        if(epoch % 10 == 0):
            print(f'Epoch {epoch:>3} | Train Loss: {total_loss/len(train_loader):.3f} | Train Acc: '
                  f'{acc/len(train_loader)*100:>6.2f}%')
          
    return model, h, out

In [ ]:
import time

# Create GCN model
gcn = GCN()
print(gcn)
print()

# Train and test
# Train
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

start_time = time.process_time()
gcn_model, gcn_output, final_output = train(gcn.to(device), data.to(device), train_loader, device)
end_time = time.process_time()

elapsed_time = end_time - start_time

print()
print('Elapsed Time (CPU): ', elapsed_time, 'seconds')

In [ ]:
print(device)